In [1]:
#import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import keras
keras.__version__

'2.14.0'

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu.name, 'memory growth: ',tf.config.experimental.get_memory_growth(gpu))

In [3]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

13568290/13568290 [==============================] - 1s 0us/step


In [4]:
df = pd.read_csv(csv_path)

Spójrzmy na dane.

In [5]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


## Zadania do wykonania. Prognozowanie sekwencji.
W prognozowaniu sekwencji, mając punkty pomiarowe z przeszłości, model musi nauczyć się przewidzieć __zakres__ przyszłych wartości. 

Model powinien przewidzieć temperaturę dla kolejnych kilku/kilkunastu pomiarów. Pozostawiam do wyboru ilość pomiarów na dobę (STEP) oraz ilość cech modelu braną do trenowania.
1. Przygotuj odpowiednio dane - w tym celu napisz odpowiedni generator. Przetestuj generator generując odpowiednie rysunki.

#### Poniżej generator z książki F. Cholleta.
Obecny generator danych zwraca krotki (samples, targets), w których obiekt samples jest wsadem danych wejściowych, a obiekt targets jest tablicą docelowych temperatur. Generator przyjmuje następujące argumenty:

* `data` — oryginalna tablica danych zmiennoprzecinkowych
* `lookback` — liczba kroków czasu, do których powinny sięgać dane wejściowe;
* `delay` — liczba kroków czasu określających przyszłą temperaturę;
* `min_index` i `max_index` — indeksy tablicy `data` określające kroki, których próbki należy odczytać. Rozwiązanie to przydaje się w celu utworzenia zbioru walidacyjnego i zbioru testowego;
* `shuffle` — określa to czy próbki mają być losowane, czy pobierane w kolejności chronologicznej;
* `batch_size` — liczba próbek wchodzących w skład wsadu;
* `step` — przedział czasu określający częstotliwość próbkowania danych. Parametrowi temu przypisujemy wartość 6, aby dysponować próbką danych odczytanych co godzinę.

In [6]:

def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets


In [14]:
def create_time_steps(length):
    return list(range(-length,0))

In [15]:
def show_plot(plot_data, delta,title):
    labels = ["Historia pomiarow", "Prawdziwa wartosc", "Prognozowana wartosc"]
    marker = [".-","rx","go"]
    time_steps = create_time_steps(plot_data[0].shape[0])
    
    if delta:
        future =delta
    else:
        future = 0
    
    plt.title(title)
    
    for i,x in enumerate(plot_data):
        if i:
            plt.plot(future,plot_data[i].flatten(), marker[i], markersize=10, label=labels[i])
            
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
        
    plt.legend()
    plt.xlim([time_steps[0], (future+5) * 2])
    plt.xlabel("Czas")
    return plt

In [9]:
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows), delay // step, data.shape[-1]))

        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            target_indices = range(rows[j], rows[j] + delay, step)

            samples[j] = data[indices]
            targets[j] = data[target_indices]

        yield samples, targets


In [11]:
lookback = 1440  # Ilość punktów z przeszłości używanych do przewidywania
delay = 144       # Ilość punktów do przewidzenia w przyszłości
batch_size = 128  # Rozmiar wsadu danych

# Definiujesz indeksy dla zbioru treningowego i testowego
train_min_index = 0
train_max_index = 200000
test_min_index = 200001
test_max_index = None  # Zakładamy, że dane testowe kończą się na ostatnim dostępnym indeksie
data = df

# Tworzysz generatory dla zbioru treningowego i testowego
train_gen = generator(data,
                      lookback=lookback,
                      delay=delay,
                      min_index=train_min_index,
                      max_index=train_max_index,
                      shuffle=True,
                      batch_size=batch_size)

test_gen = generator(data,
                     lookback=lookback,
                     delay=delay,
                     min_index=test_min_index,
                     max_index=test_max_index,
                     batch_size=batch_size)

# Ilość kroków na epokę
train_steps = (train_max_index - lookback) // batch_size
test_steps = (len(data) - test_min_index - lookback) // batch_size

In [16]:
for i in range(3):
    index = np.random.randint(0, batch_size)
    plot = show_plot(samples[index][:,1], targets[index], [0], 'Pojedyncza probka')
    plot.show()

NameError: name 'samples' is not defined